# Case 3: Understand decltype

## Key idea:
 
    decltype almost always parrots back the type of the name or expression you give it without any modification.
 

In [1]:
#include <cstddef>
#include <iostream>

class Widget {};

const int i = 0;           // decltype(i) is const int

In [2]:
bool f(const Widget& w){return false;}   // decltype(w) is const Widget&
                           // decltype(f) is bool(const Widget&)

In [3]:
struct Point {
  int x, y;                // decltype(Point::x) is int
};                         // decltype(Point::y) is int

Widget w;                  // decltype(w) is Widget

In [4]:
template<typename T>       // simplified version of std::vector
class vector {
    T dummy[1];
public:
  // ...
  T& operator[](std::size_t index){return dummy[index];}
  // ...
};

In [5]:
{

  if (f(w)) {}              // decltype(f(w)) is bool

  vector<int> v;            // decltype(v) is vector<int>
  // ...
  if (v[0] == 0) {}         // decltype(v[0]) is int&

}

 ## Key idea:
 
    Example that shows how to use decltype to compute the return type.
    Needs refinement though!


In [6]:
void authenticateUser() {}

In [7]:
template<typename Container, typename Index>  // works, but
auto authAndAccess11(Container& c, Index i)     // requires
  -> decltype(c[i])                           // refinement
{
  authenticateUser();
  return c[i];
}


## Key idea:
 
  This code attempts to assign 10 to an rvalue int, which is forbidden in C++,
  so the code won't compile.


In [8]:
#include <deque>

{
  std::deque<int> d;
  
  //authAndAccess11(d, 5) = 10;  // authenticate user, return d[5],
                             // then assign 10 to it;
                             // this won't compile!
}


## Key idea:
 
    In C++14 we can omit the trailing return type, leaving just the leading
    auto.  With that form of declaration, auto does mean that type deduction
    will take place.  In particular, it means that compilers will deduce the
    function's return type from the function's implementation.


In [9]:
template<typename Container, typename Index>  // C++14 only, and
auto authAndAccess14(Container& c, Index i)     // not quite
                                              // correct
{
  authenticateUser();
  return c[i];              // return type deduced from c[i]
}


## Key idea:
 
  This code attempts to assign 10 to an rvalue int, which is forbidden in C++,
  so the code won't compile.
 

In [10]:
#include <deque>

{
  std::deque<int> d;
  
  //authAndAccess14(d, 5) = 10;  // authenticate user, return d[5],
                             // then assign 10 to it;
                             // this won't compile!
}


## Key idea:
 
    Sometimes, one needs decltype type deduction rules in cases where types are
    inferred.  C++14 makes this possible through the decltype(auto) specifier:
    auto specifies that the type is to be deduced, and decltype says that
    decltype rules should be used during the deduction.


In [11]:
template<typename Container, typename Index>  // C++14 only;
decltype(auto)                                // works, but
authAndAccess(Container& c, Index i)          // still requires
{                                             // refinement
  authenticateUser();
  return c[i];
}


## Key idea:
 
    The use of decltype(auto) is not limited to function return types.  It can
    also be convenient for declaring variables when you want to apply the
    decltype type deduction rules to the initializing expression.


In [14]:
//class Widget {};
{
Widget w;

const Widget& cw = w;

auto myWidget1 = cw;            // auto type deduction:
                                // myWidget1's type is Widget

decltype(auto) myWidget2 = cw;  // decltype type deduction:
                                // myWidget2's type is
                                //   const Widget&
}


## Key idea:
 
    The C++11 version is almost the same as its C++14 counterpart, except that
    you have to specify the return type yourself.
 

In [15]:
#include <utility>

template<typename Container, typename Index>            // final
auto                                                    // C++11
authAndAccess11Final(Container&& c, Index i)                   // version 
-> decltype(std::forward<Container>(c)[i])
{
  authenticateUser();
  return std::forward<Container>(c)[i];
}


## Key idea:
 
    Supporting to pass rvalue containers to this function means we need to
    revise the declaration for c to accept both lvalues and rvalues, and that
    means that c needs to be a universal reference.


In [16]:
#include <utility>

template<typename Container, typename Index>            // final
decltype(auto)                                          // C++14
authAndAccess14Final(Container&& c, Index i)                   // version 
{
  authenticateUser();
  return std::forward<Container>(c)[i];
}


## Key idea:
 
    Supporting such use means we need to revise the declaration for c to accept
    both lvalues and rvalues, and that means that c needs to be a universal
    reference.
 

In [ ]:
#include <string>
#include <deque>

std::deque<std::string> makeStringDeque()   // factory function
{
  std::deque<std::string> ds;
  return ds;
}

{
  
  // make copy of 5th element of deque returned
  // from makeStringDeque
  auto s = authAndAccess11Final(makeStringDeque(), 5);
}


## Key idea:
 
    In C++14, a seemingly trivial change in the way you write a return
    statement can affect the deduced type for a function.
 

In [ ]:
decltype(auto) f1()
{
  int x = 0;
  // ...
  return x;  // decltype(x) is int, so f1 returns int
}

decltype(auto) f2()
{
  int x = 0;
  // ...
  return (x);  // decltype((x)) is int&, so f2 returns int&
}


## Key idea:
 
  For lvalue expressions more complicated than names, decltype ensures that the
  type reported is always an lvalue reference.  That is, if an lvalue
  expression other than a name has type T, decltype reports that type as T&.
 

In [ ]:
{
  int x = 0;  // decltype(x) is int
              // decltype((x)) is int&
}

                               Things to Remember
                               ------------------

* decltype almost always yields the type of a variable or expression without any
  modifications.

* For lvalue expressions of type T other than names, decltype always reports a
  type of T&.

* C++14 supports decltype(auto), which, like auto, deduces a type from its
  initializer, but it performs the type deduction using the decltype rules.

